## 这是一个没有怎么改进的RAG流程
- 使用了自己的数据集
- 没有使用任何的微调
- 没有使用任何的prompt
- 没有使用任何的模型
并且并没有很好的处理检索以及prompt等问题

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os 
import json


# 1. 导入LLM
# 初始化模型
chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称
)



## 1 构建向量库
- 利用FAISS构建向量库

In [3]:

from langchain_community.embeddings import DashScopeEmbeddings
from langchain.vectorstores import FAISS
import json
from langchain.schema import Document



In [5]:
## 创建嵌入模型
embed_model = DashScopeEmbeddings(
    model = "text-embedding-v3"
)
def load_jsonl_as_docs(jsonl_path):
    docs = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            # 拼接文本
            text = (
                f"小区名称：{data.get('小区名称', '')}；"
                f"小区单价：{data.get('小区均价', '')}；"
                f"总价：{data.get('价格', '')}；"
                f"区域：{data.get('区域', '')}；"
                f"商圈：{data.get('商圈', '')}；"
                f"其他描述：{data.get('小区解读_其他描述', '')}；"
                f"小区不足：{data.get('小区解读_小区不足', '')}；"
                f"小区户型：{data.get('小区解读_小区户型', '')}；"
                f"小区设施：{data.get('小区解读_小区设施', '')}；"
                f"居民素质：{data.get('小区解读_居民素质', '')}；"
                f"房屋品质：{data.get('小区解读_房屋品质', '')}；"
                f"生活配套：{data.get('小区解读_生活配套', '')}；"
                f"轨道交通：{data.get('小区解读_轨道交通', '')}\n"
            )
            doc = Document(page_content=text)
            docs.append(doc)
    return docs

# 用法
docs = load_jsonl_as_docs('/Users/zhuyq0719/Documents/文稿 - Zhuyq0719的MacBook Air/小区推荐系统RAG/data/processed_data.jsonl')




In [12]:
# 保存和加载向量库
# 假设你已经创建了FAISS索引
# db = FAISS.from_documents(docs, embed_model)

# # 保存索引到指定路径
# index_path = './faiss_index'
# db.save_local(index_path)


from langchain.vectorstores import FAISS
index_path = '/Users/zhuyq0719/Documents/文稿 - Zhuyq0719的MacBook Air/小区推荐系统RAG/data/faiss_index'
# 加载索引
db = FAISS.load_local(index_path, embed_model, allow_dangerous_deserialization=True)


In [13]:
# 进行serach
query = "介绍一下怡水豪庭"
result = db.similarity_search(query, k=5)
# 拼接搜索结果
content = "\n".join([doc.page_content for doc in result])

In [15]:

prompt = f"""请结合下方“小区资料”，针对用户的问题，优先推荐3个最符合条件的小区，并分别说明推荐理由。  
另外，请再推荐1个虽然不完全满足主要条件、但有一定优势、可以作为备选的小区，并说明其优缺点。

请按照以下格式输出：
1. 推荐小区1（推荐理由）,
2. 推荐小区2（推荐理由）,
3. 推荐小区3（推荐理由）,
4. 备选小区（推荐理由，说明其不足或不完全符合条件的地方）
注意，不要照着小区资料输出，可以根据用户问题着重讲解，
小区资料：
{content}

用户问题：
{query}
"""

In [16]:
print(prompt)

请结合下方“小区资料”，针对用户的问题，优先推荐3个最符合条件的小区，并分别说明推荐理由。  
另外，请再推荐1个虽然不完全满足主要条件、但有一定优势、可以作为备选的小区，并说明其优缺点。

请按照以下格式输出：
1. 推荐小区1（推荐理由）,
2. 推荐小区2（推荐理由）,
3. 推荐小区3（推荐理由）,
4. 备选小区（推荐理由，说明其不足或不完全符合条件的地方）
注意，不要照着小区资料输出，可以根据用户问题着重讲解，
小区资料：
小区名称：怡水豪庭；小区单价：43633；总价：610.0；区域：黄浦；商圈：人民广场；其他描述：None；小区不足：小区一栋楼，无绿化，距离地铁有点远；小区户型：热门户型为76方的两居室房源。楼间距90米，视野好，采光佳，空气对流，带五米露台。；小区设施：小区的1楼有健身房，住户可以免费使用，另外空中有一个小花园，可以供住户休息；居民素质：自住率比较低，大多数都是为租客。租客以老外为主。；房屋品质：07年统一装修交房，双层真空玻璃，隔音效果不错。；生活配套：None；轨道交通：None

小区名称：恒盛豪庭；小区单价：55634；总价：580.0；区域：宝山；商圈：高境；其他描述：None；小区不足：有个红绿灯路口通火车，车流量比较大，高峰期容易拥堵；小区户型：恒盛豪庭主推85和92的双南两房 ，一梯两户的中间套，103和107的南北两房边套！；小区设施：电梯直达带下车库，小区花园有两个，车位配比1:1停车场两个入口两个出口，四个进出的们；居民素质：None；房屋品质：None；生活配套：周边配套商场一个长江国际，大型超市一个永辉超市，电影院有一个娱乐方便，对面有KTV一个价格平民；轨道交通：出门多路公交，608路，1000米到达地铁站，3号线殷高西路，

小区名称：恒盛湖畔豪庭(88弄)；小区单价：41603；总价：620.0；区域：奉贤；商圈：南桥；其他描述：None；小区不足：小区价格偏高，有些顶楼墙面有少许漏水；小区户型：小区由28栋高层组成，两梯四户，112㎡丶137㎡与140㎡的大户型3房，楼间距50米，采光很好；小区设施：小区车位充足，目前地下车位已经可以停车，小区内有篮球场 网球场 喷泉 塑胶跑道饭后可娱乐；居民素质：None；房屋品质：None；生活配套：小区南面500米永辉超市，东面是政府400亿打造的上海之鱼，

### 2 构建Prompt模版

In [17]:
response = chatLLM.invoke(prompt, stream=True)
print(response)

content='1. 怡水豪庭（推荐理由：小区地理位置优越，位于黄浦区人民广场商圈，周边生活便利。热门户型为76方的两居室房源，楼间距90米，视野好，采光佳，空气对流，并且带五米露台，居住舒适度较高。）\n2. 恒盛湖畔豪庭(88弄)（推荐理由：虽然不在同一个区域，但恒盛湖畔豪庭(88弄)价格相对怡水豪庭稍低，同时拥有较好的生活配套，南面500米有永辉超市，东面是政府400亿打造的上海之鱼，3公里之内可到达百联购、苏宁易购，近商圈。此外，出门150米就有地铁五号线（金海湖路站），交通便利。）\n3. 宜浩佳园（推荐理由：宜浩佳园总价较低，仅为130.4万元，适合预算有限的购房者。小区距离滴水湖地铁站仅1公里，门口10米有公交站台，出行较为方便。小区1公里范围之内有港城广场和港城新天地商业广场，生活配套齐全。）\n\n4. 恒盛豪庭（推荐理由：恒盛豪庭的总价为580万元，低于怡水豪庭的610万元，主推85和92的双南两房，户型设计合理。不足之处在于，该小区位于宝山区高境商圈，距离市区较远，且出门需乘坐多路公交，1000米才能到达地铁站，交通不够便捷。）' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-plus'} id='run-d8de62b1-e527-4928-af54-3ca25633e13a'
